# Faster R-CNN for Object Detection and Classification

In [ ]:
!pip install torch torchvision

In [ ]:
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import Image
import matplotlib.pyplot as plt


In [ ]:
# Cargar el modelo preentrenado
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()  # Cambiar a modo de evaluación


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 88.4MB/s]


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("drive/Shared drives/Detección de polinizadores/")

In [ ]:
dict_final = {'name_img' : [], 'number_boxes': [], 'status': []}

In [ ]:
def pred_with_torch_model(image_path):
  image = Image.open(image_path).convert("RGB")
  image_tensor = F.to_tensor(image).unsqueeze(0)   # Convertir a tensor y añadir dimensión batch

    # Hacer predicción
  with torch.no_grad():
      predictions = model(image_tensor)

  # Extraer las cajas delimitadoras, etiquetas y puntuaciones
  boxes = predictions[0]['boxes']
  labels = predictions[0]['labels']
  scores = predictions[0]['scores']

  # Dibujar las cajas delimitadoras en la imagen
  #image_np = image_tensor.squeeze().permute(1, 2, 0).numpy()

  #plt.figure(figsize=(12, 8))
  #plt.imshow(image_np)

  # Filtrar las detecciones con una puntuación alta (por ejemplo, >0.5)
  threshold = 0.5

  dict_temp = {'xmin': [], 'ymin': [], 'xmax': [], 'ymax': []}

  for box, score, label in zip(boxes, scores, labels):
      if score > threshold:
          xmin, ymin, xmax, ymax = box

          dict_temp['xmin'].append(float(xmin))
          dict_temp['ymin'].append(float(ymin))
          dict_temp['xmax'].append(float(xmax))
          dict_temp['ymax'].append(float(ymax))


          #plt.gca().add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin,
                                            #edgecolor='red', facecolor='none', linewidth=2))

  #plt.axis('off')
  #plt.show()

  return dict_temp


In [ ]:
!ls

Código	Datos  Documentos  iNaturalist	Literatura  Presentaciones


In [ ]:
import pandas as pd
import numpy as np
import glob

In [ ]:
carpeta = 'Lepidoptera'

In [ ]:
path_preds = f'Datos/iNaturalist-YOLO/{carpeta}/predictions.csv'
yolo_pred = pd.read_csv(path_preds, header=None)
yolo_pred.columns = ['name_img', 'class', 'prob']
print(yolo_pred.shape)
yolo_pred.head()

(8762, 3)


,name_img,class,prob
0,Hymenoptera_1000.jpg,donut,0.26
1,Hymenoptera_100003.jpg,apple,0.35
2,Hymenoptera_100003.jpg,car,0.85
3,Hymenoptera_100040.jpg,bird,0.25
4,Hymenoptera_100040.jpg,person,0.32


In [ ]:
files_in_dir = glob.glob(f"iNaturalist/{carpeta}/*")
len(files_in_dir)

13973

In [ ]:
files_in_dir[0][24:]

'Hymenoptera_99924.jpg'

In [ ]:
len_t = 24

In [ ]:
filtered_list = list(set([x[len_t:] for x in files_in_dir]).difference(yolo_pred.name_img.unique().tolist()))
len(filtered_list)

8000

In [ ]:
filtered_list = [f'iNaturalist/{carpeta}/'+ x for x in filtered_list]
filtered_list[0]

'iNaturalist/Hymenoptera/Hymenoptera_19375.jpg'

In [ ]:
import pickle
filtered_list = pickle.load(open(f"Datos/iNaturalist-FRCNN/{carpeta}/{carpeta}_filtered_list.pkl", "rb"))
print(filtered_list[0])
print(len(filtered_list))

iNaturalist/Lepidoptera/Lepidoptera_26267.jpg
23804


In [ ]:
filtered_list.index('iNaturalist/Lepidoptera/Lepidoptera_78054.jpg')

13727

In [ ]:
for image_path in filtered_list[19000:20000]:

  try:
    dict_temp = pred_with_torch_model(image_path)

    if len(dict_temp['xmin']) == 0:
      dict_final['name_img'].append(image_path[len_t:])
      dict_final['number_boxes'].append(0)
      dict_final['status'].append('no_detection')
      print(image_path, 'no detection')
      continue

    else:
      df_temp = pd.DataFrame(dict_temp)
      df_temp.to_csv(f'Datos/iNaturalist-FRCNN/{carpeta}/'+image_path[len_t:-4] + '.txt', index=False)

      dict_final['name_img'].append(image_path[len_t:])
      dict_final['number_boxes'].append(len(dict_temp['xmin']))
      dict_final['status'].append('ok')
      print(image_path, 'detection ok')


  except:
    dict_final['name_img'].append(image_path[len_t:])
    dict_final['number_boxes'].append(np.NaN)
    dict_final['status'].append('error')

    print(image_path, 'error')


In [ ]:
pd.DataFrame(dict_final).to_csv(f'Datos/iNaturalist-FRCNN/{carpeta}/summary_20.csv', index=False)